In [23]:
# Load json file and convert it to a pandas dataframe

import json
import pandas as pd
import os
import random
import numpy as np




In [33]:
# Load all the json files at specific directory


directory = 'uncombined-training-data/time-freq-1-mil/1-milisecond-1/'
allPositiveFiles = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(directory + filename) as f:
            # df.add(json.load(f))
            allPositiveFiles.append(json.load(f))
    else:
        continue


allDataframes = []
for file in allPositiveFiles:
    tempDf = pd.DataFrame(file)
    for index, row in tempDf.iterrows():
        for index2, col in row.iteritems():
            if col != None:
                dictionaryToArray = []
                for i in col.values():
                    dictionaryToArray.append(float(i))
                tempDf.at[index, index2] = dictionaryToArray
            else:
                tempDf.at[index, index2] = [0,0,0]
    allDataframes.append(tempDf)

# combine all the dataframes into one
df = pd.concat(allDataframes, ignore_index=True)

# add one collumn dataframe at the begging of the dataframe
df.insert(0, 'Y', 1)

# find the row with the most non zero values
maxNonZero = 0
for index, row in df.iterrows():
    temp = 0
    for index2, col in row.iteritems():
        if index2 != 'Y':
            if col != [0,0,0]:
                temp += 1
    if temp > maxNonZero:
        maxNonZero = temp

# find the row with the most zero values
maxZero = 0
for index, row in df.iterrows():
    temp = 0
    for index2, col in row.iteritems():
        if index2 != 'Y':
            if col == [0,0,0]:
                temp += 1
    if temp > maxZero:
        maxZero = temp

minNonZero = maxNonZero - maxZero

# load all the negative json files at specific directory
negativeDirectory = 'uncombined-training-data/time-freq-1-mil/1-milisecond-0/'
allNegativeFiles = []
for filename in os.listdir(negativeDirectory):
    if filename.endswith(".json"):
        with open(negativeDirectory + filename) as f:
            allNegativeFiles.append(json.load(f))
    else:
        continue


allNegativeDataframes = []
for file in allNegativeFiles:
    tempDf = pd.DataFrame(file)
    for index, row in tempDf.iterrows():
        for index2, col in row.iteritems():
            if col != None:
                dictionaryToArray = []
                for i in col.values():
                    dictionaryToArray.append(float(i))
                tempDf.at[index, index2] = dictionaryToArray
            else:
                tempDf.at[index, index2] = [0,0,0]
    allNegativeDataframes.append(tempDf)

# combine all the dataframes into one
negativeDf = pd.concat(allNegativeDataframes, ignore_index=True)

print("how many rows in negative df: " + str(len(negativeDf)))



temp = []
for index, row in negativeDf.iterrows():
    split_rows = []
    split_index = random.randint(minNonZero, maxNonZero)
    start_index = 0
    while split_index < len(row.values):
        temp_row = row.values[start_index:split_index]
        start_index = split_index
        split_index = start_index + random.randint(minNonZero, maxNonZero)

        # check if all the collumns in the row are zero
        allZero = True
        for i in temp_row:
            if i != [0,0,0]:
                allZero = False
                break
        all_empty = True
        for i in temp_row:
            # if type(i) == list:
            #     for j in i:
            #         if type(j) != float and j != 0:
            #             print("not float")
            #             print(j)
            
            print(i)
            print(type(i))

            if i == 0:
                # remove that item from the list
                print("in here")
                temp_row.remove(i)
                
            # if i != 0:
            #     all_empty = False
            #     break    
        
        if allZero == False and all_empty == False:
            temp.append(temp_row)

# convert the list to numpy array

    
expanded_negativeDf = pd.DataFrame(temp)


expanded_negativeDf.insert(0, 'Y', 0)

expanded_negativeDf.to_csv('negative.tsv', sep='\t', index=False)


df = pd.concat([df, expanded_negativeDf], ignore_index=True)


print("Max non zero: " + str(maxNonZero))

print("Max zero: " + str(maxZero))


# df.to_csv('data.csv', index=False)

# Save dataframe to tab separated file
df.to_csv('data.tsv', sep='\t', index=False)


# # Load csv file
# df = pd.read_csv('data.csv')

# # Print dataframe
# df  

# # read the tsv file
# df = pd.read_csv('data.tsv', sep='\t')

# # Print dataframe
# df